In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [6]:
API_KEY = 'ad1fec0b'
#API_KEY = '4012e32e'
url = 'http://www.omdbapi.com/?apikey='+API_KEY
path = "../data/raw/"

In [3]:
df_movies = pd.read_csv(path+"movies.csv")
df_omdB = pd.read_csv(path+"omdB.csv")
failed_titles = []

In [4]:
def pullDatabyTitle(url,title,df):
    response = requests.get(url+'&t='+title)
    data = json.loads(response.text)
    return data
def pullDatabyID(url,omdbid,df):
    response = requests.get(url+'&i='+str(omdbid))
    data = json.loads(response.text)
    return data

In [ ]:
for i in df_movies['title']:
    if not (i in df_omdB['Title'].values):
        data = pullDatabyTitle(url,i,df_omdB) 
        if data['Response'] == 'True':
            df_omdB=df_omdB.append(data, ignore_index=True)
        else:
            failed_titles.append(i)
            if data['Error'] == 'Request limit reached!':
                print('Request limit reached!')
                print(i)
                break
print('Finished')

In [ ]:
failed_titles

In [11]:
df_omdB['Title'].describe()

count      10164
unique      9849
top       #DUPE#
freq           7
Name: Title, dtype: object

In [14]:
df_omdB['imdbID'].describe()

count         10158
unique        10158
top       tt0347618
freq              1
Name: imdbID, dtype: object

In [13]:
#df_omdB=df_omdB.drop_duplicates(subset='Title')
df_omdB=df_omdB.drop_duplicates(subset='imdbID')

In [15]:
progress = len(df_omdB['imdbID'])/len(df_movies['imdbID'])*100
print('{:.2f}% already pulled'.format(progress))

99.62% already pulled


In [8]:
def transformToOmdbID(x):
    switch = {
        3:"tt0000",
        4:"tt000",
        5:"tt00",
        6:"tt0",
        7:"tt",
    }
    return switch.get(len(str(x)))+str(x)

In [9]:
#pull by OmdbID
for i in df_movies['imdbID']:
    if not (transformToOmdbID(i) in df_omdB['imdbID'].values):
        data = pullDatabyID(url,transformToOmdbID(i),df_omdB) 
        if data['Response'] == 'True':
            df_omdB=df_omdB.append(data, ignore_index=True)
        else:
            failed_titles.append(i)
            if data['Error'] == 'Request limit reached!':
                print('Request limit reached!')
                print(i)
                break
print('Finished')


Finished


In [16]:
#save
df_omdB=df_omdB.drop_duplicates(subset='imdbID')
df_omdB.to_csv(path+"omdB.csv",index=False)